# Dataset Preprocessing

Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Deep Learning Libraries
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Features and Target

In [ ]:
dataset = pd.read_csv("Churn_Modelling.csv")

In [ ]:
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

Encode Categorical Data

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoder = LabelEncoder()

In [ ]:
# Encode Gender
x[:, 2] = encoder.fit_transform(x[:, 2])

In [ ]:
# Because geography is not ordinal
# And there are more than 2 values in it
# Use OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
# Drop one dummy variable to avoid Dummy Variable Trap
x = np.array(ct.fit_transform(x))
x = x[:, 1:]

Splitting the Dataset

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

Feature Scaling

In [ ]:
# Ease calculations resources
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Model Evaluation

Function to Build and Evaluate the Architecture of the ANN

In [ ]:
def build_ann_architecture():
  # Initializing Artificial Neural Networks
  classifier = Sequential()
  # Adding Input Layer
  classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform', input_dim = 11))
  # Hidden Layers
  classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))
  # Output Layers
  classifier.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))
  # Similar to applying gradiant descent to all the layers
  classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

  return classifier

K-Fold Cross Validation

In [ ]:
classifier = KerasClassifier(build_fn = build_ann_architecture, batch_size = 10, nb_epoch = 100)
accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10, n_jobs = -1)

In [ ]:
mean = accuracies.mean()
std = accuracies.std()

# Dropout Regularization

Function to Build the Architecture of the ANN

In [ ]:
def build_ann_architecture():
  # Reduce Overfitting -> High Variance, Accuracy on Training Set >>> Accuracy on Test Set
  # To reduce overfitting, use Dropout Regularization
  # to prevent the neuron learning too much from each other
  # The neuron is killed and rewritten in each iteration, never kill more than 0.5 neurons
  # Apply the regularization to all layers to avoid overfitting

  # Initializing Artificial Neural Networks
  classifier = Sequential()
  # Adding Input Layer
  classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform', input_dim = 11))
  classifier.add(Dropout(p = 0.1))
  # Hidden Layers
  classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))
  classifier.add(Dropout(p = 0.1))
  # Output Layers
  classifier.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))
  # Similar to applying gradiant descent to all the layers
  classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

  return classifier

# Tuning the ANN

Dictionary to Contain Hyperparameters

In [ ]:
hyperparams = {"batch_size": [10, 25, 32],
               "nb_epoch": [100, 250, 500],
               "optimizer": ["adam", "rmsprop"]
               }

Function to Build and Tune the Architecture of the ANN

In [ ]:
def build_ann_architecture(optimizer):
  # Initializing Artificial Neural Networks
  classifier = Sequential()
  # Adding Input Layer
  classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform', input_dim = 11))
  classifier.add(Dropout(0.2))
  # Hidden Layers
  classifier.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))
  classifier.add(Dropout(0.2))
  # Output Layers
  classifier.add(Dense(units=1, activation='relu', kernel_initializer='uniform'))
  # Similar to applying gradiant descent to all the layers
  classifier.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])

  return classifier

In [ ]:
# Remove all the hyperparameters for the tuning
classifier = KerasClassifier(build_fn = build_ann_architecture)

Use Grid Search to do Hyperparameter Tuning

In [ ]:
grid_search = GridSearchCV(estimator = classifier, param_grid = hyperparams, scoring = "accuracy", cv = 10)
# Fit the Grid Search
grid_search = grid_search.fit(x_train, y_train)

720/720 [==============================] - 1s 957us/step - loss: 0.6719 - accuracy: 0.7980


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


800/800 [==============================] - 1s 1ms/step - loss: 3.1772 - accuracy: 0.7940


Get the Best Parameters after Tuning

In [ ]:
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
best_params

{'batch_size': 10, 'nb_epoch': 500, 'optimizer': 'rmsprop'}

In [ ]:
best_accuracy

0.8026249999999999